In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from flow_package import data_preprocessing

full_features = [
    "Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets","Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min","Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std","Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count","ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes","Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd","min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label"
]

dataset_cicids2017_paths = [
    "raw/cicids2017/Benign-Monday-WorkingHours.pcap_ISCX.csv",
    "raw/cicids2017/Botnet-Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "raw/cicids2017/Bruteforce-Tuesday-WorkingHours.pcap_ISCX.csv",
    "raw/cicids2017/DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX.csv",
    "raw/cicids2017/DoS-Wednesday-WorkingHours.pcap_ISCX.csv",
    "raw/cicids2017/Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX.csv",
    "raw/cicids2017/Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX.csv",
    "raw/cicids2017/WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX.csv"
]

In [8]:
csecicids2018_features = [
    "Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets","Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min","Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std","Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count","ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes","Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd","min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label"
]

dataset_csecicids2018_paths = [
    "raw/csecicids2018/02-14-2018.csv",
    "raw/csecicids2018/02-15-2018.csv",
    "raw/csecicids2018/02-16-2018.csv",
    "raw/csecicids2018/02-20-2018.csv",
    "raw/csecicids2018/02-21-2018.csv",
    "raw/csecicids2018/02-22-2018.csv",
    "raw/csecicids2018/02-23-2018.csv",
    "raw/csecicids2018/02-28-2018.csv",
    "raw/csecicids2018/03-01-2018.csv",
    "raw/csecicids2018/03-02-2018.csv",
]

In [ ]:
def create_dataset(paths, features, name):
    TRAIN_PATH = f"dataset/{name}/train.csv"
    TEST_PATH = f"dataset/{name}/test.csv"

    BINARY_PATH = f"dataset/{name}/binary"

    MULTIPLE_PATH = f"dataset/{name}/multi"

    N = 5000

    def func_binary():
        buf_train, buf_test, label_list = data_preprocessing(
            train_data=TRAIN_PATH,
            test_data=TEST_PATH,
            categorical_index=["Protocol", "Destination Port"],
            binary_normal_label="BENIGN"
        )
        
        print("\tstart write train csv")
        for i in range(0, len(buf_train), N):
            binary_train = buf_train[i:i+N]
            binary_train.to_csv(f"{BINARY_PATH}/train/train_{i}.csv")

        print("\tstart write test csv")
        for i in range(0, len(buf_test), N):
            binary_test = buf_test[i:i+N]
            binary_test.to_csv(f"{BINARY_PATH}/test/test_{i}.csv")

        with open(f"{BINARY_PATH}/label.txt", "w") as f:
            f.write("\n".join(label_list))
    
    def func_multiple():
        buf_train, buf_test, label_list = data_preprocessing(
            train_data=TRAIN_PATH,
            test_data=TEST_PATH,
            categorical_index=["Protocol", "Destination Port"],
        )

        print("\tstart write train csv")
        for i in range(0, len(buf_train), N):
            multiple_train = buf_train[i:i+N]
            multiple_train.to_csv(f"{MULTIPLE_PATH}/train/train_{i}.csv")

        print("\tstart write test csv")
        for i in range(0, len(buf_test), N):
            multiple_test = buf_test[i:i+N]
            multiple_test.to_csv(f"{MULTIPLE_PATH}/test/test_{i}.csv")

        with open(f"{MULTIPLE_PATH}/label.txt", "w") as f:
            f.write("\n".join(label_list))


    # empty dataframe
    train = pd.DataFrame()
    test = pd.DataFrame()

    for path in paths:
        # print absolute path
        abs_path = os.path.abspath(path)
        # print(os.path.abspath(path))
        # read csv
        df = pd.read_csv(abs_path)
        if len(df.columns) == 84:
            df = df.set_axis(full_features, axis=1)
            df = df.drop(columns=["Flow ID", "Source IP", "Source Port", "Destination IP"])
        else:
            df = df.set_axis(features, axis=1)
        # split data
        df_b, df_a = train_test_split(df, test_size=0.01)
        # print(f"before: {len(df_b)} after: {len(df_a)}")
        train_df, test_df = train_test_split(df_a, test_size=0.2)

        train_df = train_df.reset_index(drop=True)
        test_df = test_df.reset_index(drop=True)

        # append data
        train = pd.concat([train, train_df])
        test = pd.concat([test, test_df])
        print(f"train: {len(train_df)} test: {len(test_df)}")
        print(f"{path}: done")
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    # print(f"train: {len(train)} test: {len(test)}")

    train.to_csv(TRAIN_PATH)
    test.to_csv(TEST_PATH)

    print("start binary")
    func_binary()
    print("start multiple")
    func_multiple()

In [ ]:
create_dataset(dataset_cicids2017_paths, full_features, "cicids2017")

train: 4240 test: 1060
raw/cicids2017/Benign-Monday-WorkingHours.pcap_ISCX.csv: done
train: 1528 test: 383
raw/cicids2017/Botnet-Friday-WorkingHours-Morning.pcap_ISCX.csv: done
train: 3568 test: 892
raw/cicids2017/Bruteforce-Tuesday-WorkingHours.pcap_ISCX.csv: done
train: 1806 test: 452
raw/cicids2017/DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX.csv: done
train: 5542 test: 1386
raw/cicids2017/DoS-Wednesday-WorkingHours.pcap_ISCX.csv: done
train: 2309 test: 578
raw/cicids2017/Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX.csv: done
train: 2292 test: 573
raw/cicids2017/Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX.csv: done
train: 1363 test: 341
raw/cicids2017/WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX.csv: done
train: 22648 test: 5665
--------------cic ids 2017-----------------------
-----------------done------------------------


In [ ]:
create_dataset(dataset_csecicids2018_paths, csecicids2018_features, "csecicids2018")

train: 8388 test: 2098
raw/csecicids2018/02-14-2018.csv: done
train: 8388 test: 2098
raw/csecicids2018/02-15-2018.csv: done


/var/folders/d3/7m94kh9j1xg5sc7cx2lk48t00000gn/T/ipykernel_5539/3872305196.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_path)


train: 8388 test: 2098
raw/csecicids2018/02-16-2018.csv: done
train: 63590 test: 15898
raw/csecicids2018/02-20-2018.csv: done
train: 8388 test: 2098
raw/csecicids2018/02-21-2018.csv: done
train: 8388 test: 2098
raw/csecicids2018/02-22-2018.csv: done
train: 8388 test: 2098
raw/csecicids2018/02-23-2018.csv: done


/var/folders/d3/7m94kh9j1xg5sc7cx2lk48t00000gn/T/ipykernel_5539/3872305196.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_path)


train: 4905 test: 1227
raw/csecicids2018/02-28-2018.csv: done


/var/folders/d3/7m94kh9j1xg5sc7cx2lk48t00000gn/T/ipykernel_5539/3872305196.py:11: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_path)


train: 2649 test: 663
raw/csecicids2018/03-01-2018.csv: done
train: 8388 test: 2098
raw/csecicids2018/03-02-2018.csv: done
train: 129860 test: 32474
---------------cse cic ids 2018---------------------
-----------------done----------------
